In [1]:
import numpy as np
from matplotlib import pyplot as plt
import glob
import os
from skimage import io
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import sys

tf.test.gpu_device_name()
print (tf.__version__)
tf.config.experimental.list_physical_devices(device_type=None)

2.4.0


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
# generic way to create custom callback
class LossHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        
        


In [3]:
#datasets = ['Orig', 'Image_th', 'Filter_RGB', 'Processed']
datasets = [  'all_orig']
#Models = [  'Resnet', 'CheXNet','Dense','Inception', 'VGG']
Models = [  'Dense','Inception', 'VGG', 'CheXNet']
losses = {'Dense-Orig':[], 'Dense-Image_th':[], 'Dense-Filter_RGB':[], 'Dense-Processed':[], 
         'VGG-Orig':[], 'VGG-Image_th':[], 'VGG-Filter_RGB':[], 'VGG-Processed':[], 
         'Inception-Orig':[], 'Inception-Image_th':[], 'Inception-Filter_RGB':[], 'Inception-Processed':[], 
         'Resnet-Orig':[], 'Resnet-Image_th':[], 'Resnet-Filter_RGB':[], 'Resnet-Processed':[],
         'CheXNet-Orig':[], 'CheXNet-Image_th':[], 'CheXNet-Filter_RGB':[], 'CheXNet-Processed':[], 
         'Dense-all_orig':[], 'VGG-all_orig':[], 'Inception-all_orig':[], 'Resnet-all_orig':[], 'CheXNet-all_orig':[]}

# specify image size and channels
img_channels = 3
img_rows = 299
img_cols = 299

# number of classes
nb_classes = 3

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import optimizers
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications import  DenseNet201 ,InceptionV3, DenseNet121, ResNet101
from tensorflow.keras.layers import Input, Add, AveragePooling2D,  GlobalAveragePooling2D ,GlobalMaxPool2D
from tensorflow.keras.callbacks import ModelCheckpoint

def get_model(model, input_shape = (img_rows,img_cols,img_channels)):
    #Get base model: ResNet50 or VGG16
    finetune = True
    chex = False
    if model == 'Dense':
        base_model =  DenseNet201(weights='imagenet', include_top=False ,input_shape=input_shape) 
        split_at = 481
    elif model == 'VGG':
        base_model =  VGG19(weights='imagenet', include_top=False ,input_shape=input_shape)
        split_at = -8
        finetune = False
    elif model == 'Inception':
        base_model =  InceptionV3(weights='imagenet', include_top=False ,input_shape=input_shape) 
        split_at = 249
    elif model == 'Resnet':
        base_model =  ResNet101(weights='imagenet', include_top=False ,input_shape=input_shape)
        split_at = 313
    elif model == 'CheXNet':
        base_model = DenseNet121( include_top=False, weights=None, input_shape=input_shape )
        split_at = 313
        chex = True
    else :
        return 'No model'


    # Get the output from the base model 
    base_model_output = base_model.output

    x = base_model.output
    x = AveragePooling2D()(x)
    x = Flatten()(x)
    predictions = Dense(3, activation= 'softmax')(x)

#     dependencies = {
#         'ValidAccuracy': ValidAccuracy
#     }

    model = Model(inputs=base_model.input, outputs=predictions)
#Finetuning
    # first: train only the top layers (which were randomly initialized)
    # i.e. freeze all convolutional InceptionV3 layers
    if finetune :
        for layer in base_model.layers:
            layer.trainable = False

        model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
        # train the model on the new data for a few epochs
        model.fit_generator(training_set,  epochs=2, validation_data=validation_set, validation_steps=21)
        if chex :
            model.load_weights('./data/brucechou1983_CheXNet_Keras_0.3.0_weights.h5',by_name=True, skip_mismatch=True)

        # let's visualize layer names and layer indices to see how many layers
        # we should freeze:
#         for i, layer in enumerate(base_model.layers):
#            print(i, layer.name)
#fine tune end============================================
    
    for layer in model.layers[:split_at]:
       layer.trainable = False
    for layer in model.layers[split_at:]:
       layer.trainable = True

    #print (model.summary())
    return model
#==========end of model creation==================================
#================Looped iteration================================
for modeltype in Models:
    for dataset in datasets:
        print ('=========================================================================')
        print ('Executing ' + str(modeltype) + ' for dataset- ' + str(dataset))
        print ('=========================================================================')
        
        # path to your dataset
        Train_DATASET_PATH = './data/'+ str(dataset) + '/train'
        TEST_DATASET = './data/'+ str(dataset) + '/test'
        data_cls = ['COVID', 'Normal', 'Viral']

        # globbing example
        # help(glob)
        #covid_path = os.path.join(Train_DATASET_PATH, data_cls[0], '*')
        print(Train_DATASET_PATH)
        print(TEST_DATASET)
        print(modeltype)


        #Create the datagenerator
        datagen = ImageDataGenerator(rescale=1. / 255, validation_split=0.2,
                                     rotation_range=20,
                                     width_shift_range=0.1,
                                     height_shift_range=0.1,
                                     shear_range=0.1,
                                     zoom_range=0.1,
                                     horizontal_flip=True)
        print (Train_DATASET_PATH)
        training_set = datagen.flow_from_directory(
                Train_DATASET_PATH,
                target_size=(299, 299),
                batch_size=32,shuffle=True,
                class_mode='categorical',
                subset='training')

        validation_set = datagen.flow_from_directory(
            Train_DATASET_PATH,
            target_size=(299, 299),
            batch_size=32,shuffle=True,
            class_mode='categorical',
            subset='validation')
        #================================================================================================
        #Get Model

        model = get_model(modeltype)
        opt = tf.keras.optimizers.SGD()
        model.compile(loss='categorical_crossentropy',optimizer=opt,
                      metrics=['accuracy'])

        # generators
        #training_generator = DataGenerator('train')#, ablation=50)
        #validation_generator = DataGenerator('val')#, ablation=50)
        #print (training_generator)

        history = LossHistory()
        plot_data = {}
        filepath = './models_img/'+str(modeltype)+'-'+str(dataset)+'.hdf5'
        checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

        # fit
        hist = model.fit_generator(training_set,  epochs=20, validation_data=validation_set, validation_steps=21, callbacks=[history, checkpoint])
        # for key in hist.history:
        #    print(key)
        #================================================================================================

        #print (len(history.losses))
        print (history.losses)
        losses[str(modeltype)+'-'+str(dataset)].append (history.losses)

        #======================================================================================================
        #load  model for test
        #using the inbuilt imagegenerator
        import h5py
        from tensorflow.keras.models import load_model
        from tensorflow.keras.preprocessing.image import ImageDataGenerator
        #from tensorflow.math import confusion_matrix
        from sklearn.metrics import classification_report, confusion_matrix

        test_datagen = ImageDataGenerator(rescale=1. / 255)
        test_set = test_datagen.flow_from_directory(
                TEST_DATASET,
                target_size=(299, 299),
                batch_size=32,shuffle=False,
                class_mode='categorical')

        import matplotlib.pyplot as plt

        test_set.reset() 

        model = load_model('./models_img/'+str(modeltype)+'-'+str(dataset)+'.hdf5')

        Y_pred = model.predict_generator(test_set,1514 // 32+1)
        y_pred = np.argmax(Y_pred,axis=-1)
        s=confusion_matrix(test_set.classes,y_pred)

        #=====================================================================================================
        #Display results for test
        from sklearn.metrics import accuracy_score
        target_names = data_cls
        print (test_set.classes)
        print (y_pred)
        c = classification_report(test_set.classes.tolist(),y_pred.tolist(),target_names=target_names)

        with open('results.txt', 'a') as f:
            print (str(modeltype) + '-' + str(dataset), file=f)
            print ('==============================================================================', file=f)
            print (s, file=f)
            print (c, file=f)
            #print ('Accuracy------ ' )
            print ('Test Accuracy------ '+ str(accuracy_score(test_set.classes.tolist(),y_pred.tolist())), file=f)
            print ('loss ------------', file=f)
            print (history.losses, file=f)
            print ('=============================End of run========================================', file=f)

Executing Dense for dataset- all_orig
./data/all_orig/train
./data/all_orig/test
Dense
./data/all_orig/train
Found 10912 images belonging to 3 classes.
Found 2727 images belonging to 3 classes.
74842112/74836368 [==============================] - 1s 0us/step


/mnt/disks/user/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/2
341/341 [==============================] - 285s 785ms/step - loss: 1.3424 - val_loss: 0.4415
Epoch 2/2
341/341 [==============================] - 264s 773ms/step - loss: 0.5577 - val_loss: 1.0684
Epoch 1/20
341/341 [==============================] - 285s 794ms/step - loss: 1.7855 - accuracy: 0.8638 - val_loss: 0.3054 - val_accuracy: 0.9286

Epoch 00001: val_accuracy improved from -inf to 0.92857, saving model to ./models_img/Dense-all_orig.hdf5
Epoch 2/20
341/341 [==============================] - 267s 783ms/step - loss: 0.1814 - accuracy: 0.9451 - val_loss: 0.3642 - val_accuracy: 0.9196

Epoch 00002: val_accuracy did not improve from 0.92857
Epoch 3/20
341/341 [==============================] - 261s 764ms/step - loss: 0.2440 - accuracy: 0.9481 - val_loss: 0.0971 - val_accuracy: 0.9702

Epoch 00003: val_accuracy improved from 0.92857 to 0.97024, saving model to ./models_img/Dense-all_orig.hdf5
Epoch 4/20
341/341 [==============================] - 267s 781ms/step - loss: 0.131

/mnt/disks/user/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


[0 0 0 ... 2 2 2]
[0 0 0 ... 2 2 2]
Executing Inception for dataset- all_orig
./data/all_orig/train
./data/all_orig/test
Inception
./data/all_orig/train
Found 10912 images belonging to 3 classes.
Found 2727 images belonging to 3 classes.
87916544/87910968 [==============================] - 1s 0us/step


/mnt/disks/user/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/2
341/341 [==============================] - 253s 722ms/step - loss: 2.9415 - val_loss: 1.0959
Epoch 2/2
341/341 [==============================] - 248s 727ms/step - loss: 1.3378 - val_loss: 1.4196
Epoch 1/20
341/341 [==============================] - 260s 745ms/step - loss: 0.8063 - accuracy: 0.8855 - val_loss: 0.5627 - val_accuracy: 0.8988

Epoch 00001: val_accuracy improved from -inf to 0.89881, saving model to ./models_img/Inception-all_orig.hdf5
Epoch 2/20
341/341 [==============================] - 254s 745ms/step - loss: 0.1827 - accuracy: 0.9484 - val_loss: 0.1239 - val_accuracy: 0.9658

Epoch 00002: val_accuracy improved from 0.89881 to 0.96577, saving model to ./models_img/Inception-all_orig.hdf5
Epoch 3/20
341/341 [==============================] - 249s 729ms/step - loss: 0.1251 - accuracy: 0.9608 - val_loss: 0.1254 - val_accuracy: 0.9613

Epoch 00003: val_accuracy did not improve from 0.96577
Epoch 4/20
341/341 [==============================] - 249s 731ms/step - los

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



341/341 [==============================] - 249s 731ms/step - loss: 0.0718 - accuracy: 0.9750 - val_loss: 0.0817 - val_accuracy: 0.9717

Epoch 00006: val_accuracy improved from 0.96577 to 0.97173, saving model to ./models_img/Inception-all_orig.hdf5
Epoch 7/20
 48/341 [===>..........................] - ETA: 3:19 - loss: 0.0778 - accuracy: 0.9738

In [ ]:
#print (losses)